In [1]:
# Load TensorBoard notebook extension
%load_ext tensorboard

Run ``tfds build --overwrite`` under lfw_pairs directory first.

# Imports

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from deepface import DeepFace

2024-01-23 21:46:49.170298: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 21:46:49.209074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 21:46:49.209097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 21:46:49.210192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 21:46:49.216644: I tensorflow/core/platform/cpu_feature_guar

In [3]:
LOG_DIR = './logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
FILE_WRITER = tf.summary.create_file_writer(LOG_DIR)

2024-01-23 21:46:51.986506: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 21:46:52.023575: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 21:46:52.025020: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Dataset

In [4]:
ds, info = tfds.load('lfw_pairs',
                      with_info=True,
                      download=True,
                      as_supervised=False)

In [5]:
bench_ds = ds['train'].batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
tfds.benchmark(bench_ds, batch_size=32)


************ Summary ************



  0%|          | 0/69 [00:00<?, ?it/s]

Examples/sec (First included) 5740.47 ex/sec (total: 2240 ex, 0.39 sec)
Examples/sec (First only) 485.10 ex/sec (total: 32 ex, 0.07 sec)
Examples/sec (First excluded) 6809.64 ex/sec (total: 2208 ex, 0.32 sec)


,duration,num_examples,avg
first+lasts,0.390212,2240,5740.472829
first,0.065966,32,485.100083
lasts,0.324246,2208,6809.644739


In [6]:
with FILE_WRITER.as_default():
    tmp = np.array([x['img_0'] for x in ds['train'].take(25).as_numpy_iterator()])
    tf.summary.image("Pair 0", tmp, max_outputs=25, step=0)
    tmp = np.array([x['img_1'] for x in ds['train'].take(25).as_numpy_iterator()])
    tf.summary.image("Pair 1", tmp, max_outputs=25, step=0)

2024-01-23 21:46:53.132247: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [7]:
info.features

FeaturesDict({
    'img_0': Image(shape=(250, 250, 3), dtype=uint8),
    'img_1': Image(shape=(250, 250, 3), dtype=uint8),
    'pair': ClassLabel(shape=(), dtype=int64, num_classes=2),
})

In [8]:
info

tfds.core.DatasetInfo(
    name='lfw_pairs',
    full_name='lfw_pairs/0.1.0',
    description="""
    TODO(lfw_pairs): Markdown description of that will appear on the catalog page.
    Description is **formatted** as markdown.
    
    It should also contain any processing which has been applied (if any),
    (e.g. corrupted example skipped, images cropped,...):
    """,
    homepage='http://vis-www.cs.umass.edu/lfw',
    data_dir='/home/coder/tensorflow_datasets/lfw_pairs/0.1.0',
    file_format=tfrecord,
    download_size=172.28 MiB,
    dataset_size=87.63 MiB,
    features=FeaturesDict({
        'img_0': Image(shape=(250, 250, 3), dtype=uint8),
        'img_1': Image(shape=(250, 250, 3), dtype=uint8),
        'pair': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=None,
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=1000, num_shards=1>,
        'train': <SplitInfo num_examples=2200, num_shards=1>,
    },
    citation="

# Using DeepFace

In [9]:
for x in ds['test'].take(1).as_numpy_iterator():
  print(DeepFace.represent(x['img_0']))

2024-01-23 21:46:53.637838: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-01-23 21:46:54.717436: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-23 21:46:54.784364: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


[{'embedding': [0.013011282309889793, -0.015226741321384907, 0.013492593541741371, 0.01778615452349186, 0.002676218282431364, -4.877801984548569e-06, -0.004543793387711048, 0.021796349436044693, 0.011312596499919891, 0.004416341427713633, 0.007855897769331932, 0.016241975128650665, 0.012072640471160412, -0.0010150950402021408, 0.01858954131603241, 0.002436863025650382, 0.010004756972193718, 0.009952010586857796, 0.009191938675940037, -0.007958070375025272, 0.009729885496199131, 0.011684591881930828, -0.004467841237783432, 0.002482574200257659, 0.0016619444359093904, 0.005379594396799803, 0.006381484679877758, -0.013566440902650356, 0.020415838807821274, 0.013238325715065002, 0.01187104731798172, 0.00190900475718081, -0.023499298840761185, 0.012398709543049335, 0.005407895892858505, 0.030712399631738663, 0.0060654026456177235, -0.006363534834235907, 0.002496185014024377, 0.02355228178203106, 0.00019508996047079563, -0.004965735599398613, -0.008340567350387573, -0.01136095356196165, -0.0

In [10]:
model = DeepFace.build_model(model_name='Facenet')

In [11]:
gt = []
pred = []

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
]

ds_test = ds['test'].batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

for x in ds['train'].take(50).as_numpy_iterator():
  pred.append(DeepFace.verify(x['img_0'], x['img_1'], model_name=models[3], enforce_detection=False)['verified'])
  gt.append(x['pair'])

In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(gt, pred))

0.54
